In [144]:
from bs4 import BeautifulSoup 
import requests
import webbrowser
from urllib.parse import urlparse
import pandas as pd
import os
import time
import re

In [145]:
def get_home_page(url, sym):
    '''Parse the Home page from ETFdb.com'''
    time.sleep(0.5)
    resp = requests.get(url = url)
    dom = resp.text
    soup = BeautifulSoup(dom, 'html.parser')
    if soup.find_all('a', string = "Home page"):
        link = str(soup.find('a', string = "Home page").attrs['href'])
        return str(link)
    else:
        print("Can't find the home url!")
    

In [175]:
def write_raw_csv(url, csv_name, results_path = os.path.join(work_dir, 'data', 'Raw_csv')):
    '''Write csv from url'''
    if not os.path.isdir(results_path):
        os.mkdir(results_path)
    response = requests.get(url)
    with open(os.path.join(results_path, csv_name),  'wb') as f:
        f.write(response.content)
        f.close()
    print("Sucessfully writing {}".format(csv_name))
    

## Parse from flexshares

In [147]:
def download_from_flexshares(link, sym):
    time.sleep(0.5)
    resp = requests.get(url = link)
    dom = resp.text
    soup = BeautifulSoup(dom, 'html.parser')
    csv_name = sym + ".csv"
    if soup.find('a', href = re.compile(".*?\.csv")):
        link_tail = soup.find('a', href = re.compile(".*?\.csv")).attrs['href']
        url = 'https://www.flexshares.com' + str(link_tail)
        write_raw_csv(url, csv_name)
    else:
        print("Can't download raw csv!")
    

## Parse from IShares

In [191]:
def download_from_ishares(link, sym):
    time.sleep(0.5)
    resp = requests.get(url = link)
    dom = resp.text
    soup = BeautifulSoup(dom, 'html.parser')
    csv_name = sym + ".xls"
    if soup.find('a', string = 'Download'):
        link_tail = soup.find('a',  class_="icon-xls-export", href = re.compile("/us/products/*"), string = 'Download').attrs['href']
        url = 'https://www.flexshares.com' + str(link_tail)
        write_raw_csv(url, csv_name)
    else:
        print("Can't download raw csv!")


## Classify links

In [192]:
def classify_web_page(link, sym, groupName):
    if groupName == 'FlexShares':
        download_from_flexshares(link, sym)
        return True
    elif groupName == 'iShares':
        download_from_ishares(link, sym)
        return True
        
    else:
        print("Can't download raw csv!")
        return False
            

# Main Function

In [193]:
if __name__ ==  '__main__':
    work_dir = os.getcwd()
    data_dir = os.path.join(work_dir, 'data', 'Input_data')
    ETFwebsite = 'https://etfdb.com/etf'
    input_data = 'group17_region.csv'
    ETFList = pd.read_csv(os.path.join(os.path.join(data_dir), input_data))
    ETF_Symbol= ETFList['Symbol']
    ETF_GroupNames= ETFList['Group']
    for sym, groupName in zip(ETF_Symbol, ETF_GroupNames):
        if groupName == 'iShares':
            url = ETFwebsite + "/" + str(sym)
            print(sym)
            link = get_home_page(url, sym)
            print(link)
            classify_web_page(link, sym, groupName)
 

IEMG
http://us.ishares.com/product_info/fund/overview/IEMG.htm?fundSearch=true&qt=IEMG
Sucessfully writing IEMG.xls
HEEM
https://www.ishares.com/us/products/268704/ishares-currency-hedged-msci-emerging-markets
Sucessfully writing HEEM.xls
AXJV
http://www.ishares.com/us/products/264606/AXJV
Sucessfully writing AXJV.xls
EMGF
https://www.ishares.com/us/products/272820/ishares-msci-emerging-multi-factor-etf
Sucessfully writing EMGF.xls
ACWX
http://us.ishares.com/product_info/fund/overview/ACWX.htm?qt=ACWX
Sucessfully writing ACWX.xls
AAXJ
http://us.ishares.com/product_info/fund/overview/AAXJ.htm?qt=AAXJ
Sucessfully writing AAXJ.xls
EEMA
https://www.ishares.com/us/products/239629/EEMA?referrer=tickerSearch
Sucessfully writing EEMA.xls
AIA
http://us.ishares.com/product_info/fund/overview/AIA.htm?qt=AIA
Sucessfully writing AIA.xls
